## Kyverno

![](https://kyverno.io/images/kyverno-architecture.png)^

Quelle: https://kyverno.io/docs/introduction/how-kyverno-works/
- - - 

Kyverno (griechisch für „regieren“) ist eine Cloud-native Policy-Engine. Sie wurde ursprünglich für Kubernetes entwickelt und kann nun auch ausserhalb von Kubernetes-Clustern als einheitliche Policy-Sprache eingesetzt werden.

Mit Kyverno können Plattformingenieure die Validierung von Sicherheit, Compliance und Best Practices automatisieren und Anwendungsteams einen sicheren Self-Service bieten.

Zu den zahlreichen Funktionen gehören unter anderem:

* Richtlinien als YAML-basierte deklarative Kubernetes-Ressourcen, ohne dass Sie eine neue Sprache lernen müssen!
* Durchsetzung von Richtlinien als Kubernetes-Zulassungscontroller, CLI-basierter Scanner und zur Laufzeit
* Validieren, mutieren, generieren oder bereinigen (entfernen) Sie jede Kubernetes-Ressource
* Überprüfen Sie Container-Images und Metadaten für die Sicherheit der Software-Lieferkette
* Richtlinien für alle JSON-Nutzdaten, einschliesslich Terraform-Ressourcen, Cloud-Ressourcen und Serviceautorisierung
* Richtlinienberichterstattung unter Verwendung des offenen Berichtsformats der CNCF Policy WG
* flexibles Richtlinienausnahmemanagement
* Werkzeuge für umfassende Unit- und E2E-Tests von Richtlinien
* Verwaltung von Richtlinien als Coderessourcen mit vertrauten Tools wie `git` und `kustomize`



Installieren Sie zunächst Kyverno

In [ ]:
%%bash
helm repo add kyverno https://kyverno.github.io/kyverno/
helm repo update
helm install kyverno kyverno/kyverno -n kyverno --create-namespace

Fügen Sie die unten stehende Richtlinie zu Ihrem Cluster hinzu. Sie enthält eine einzige Validierungsregel, die erfordert, dass alle Pods das teamLabel haben.

In [ ]:
%%bash
kubectl create -f- << EOF
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: require-labels
spec:
  rules:
  - name: check-team
    match:
      any:
      - resources:
          kinds:
          - Pod
    validate:
      failureAction: Enforce
      message: "label 'team' is required"
      pattern:
        metadata:
          labels:
            team: "?*"
EOF


Versuchen Sie, eine Bereitstellung ohne die erforderliche Bezeichnung zu erstellen.

In [ ]:
%%bash
kubectl create deployment nginx --image=nginx || true


Sie sollten einen Fehler sehen und das Deployment wurde nicht erstellt

In [ ]:
%%bash
kubectl get all

Erstellen Sie nun einen Pod mit der erforderlichen Bezeichnung.

In [ ]:
%%bash
kubectl run nginx --image nginx --labels team=backend
kubectl get all

Diese Pod-Konfiguration entspricht der Richtlinie und ist zulässig.

- - -

Nachdem der Pod nun existiert, warten Sie noch ein paar Sekunden und sehen Sie, welche weiteren Aktionen Kyverno ausgeführt hat. Führen Sie den folgenden Befehl aus, um den von Kyverno erstellten Richtlinienbericht abzurufen.

In [ ]:
%%bash
kubectl get policyreport -o wide

Und die Details dazu

In [ ]:
%%bash
kubectl describe policyreport

Regel wieder löschen

In [ ]:
%%bash
kubectl delete clusterpolicy require-labels
kubectl delete deployment nginx

## Weitere Beispiele

* [Mutationsressourcen](https://kyverno.io/docs/introduction/quick-start/#mutate-resources)